In [ ]:
# Install keras_ocr
!pip install keras-ocr

In [ ]:
# imports
import keras_ocr
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import glob
import matplotlib.image as mpimg
from collections import Counter

In [ ]:
#This function is for cropping out the LUS measurement scales. It crops RoI
def crop_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert image to grayscale

    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    close_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    close = cv2.morphologyEx(gray.astype(np.uint8), cv2.MORPH_CLOSE, close_kernel, iterations=1)

    dilate_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    dilate = cv2.dilate(close, dilate_kernel, iterations=1)

    cnts,_ = cv2.findContours(dilate.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # mask=np.zeros_like(gray)
    areas =list()
    for c in cnts:
        area = cv2.contourArea(c)
        areas.append(area)

    c_=cnts[np.argmax(areas)]
    x,y,w,h = cv2.boundingRect(c_)
    return img[y:y+h,x:x+w]

In [ ]:
# This function removes the text labels on the image
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)

def inpaint_text(img, pipeline):
    prediction_groups = pipeline.recognize([img])
    mask = np.zeros(img.shape[:2], dtype="uint8")
    for box in prediction_groups[0]:
        x0, y0 = box[1][0]
        x1, y1 = box[1][1]
        x2, y2 = box[1][2]
        x3, y3 = box[1][3]

        x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)

        thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ))

        cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255,
        thickness)
        img = cv2.inpaint(img, mask, 2, cv2.INPAINT_TELEA)

    return(img)

In [ ]:
# This function removes letter P on the image
def inpaint_P(img):
    img = np.array(img)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    ret, thresh = cv2.threshold(gray.astype('uint8'), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    close_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    close = cv2.morphologyEx(gray.astype(np.uint8), cv2.MORPH_CLOSE, close_kernel, iterations=1)

    dilate_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    dilate = cv2.dilate(close, dilate_kernel, iterations=1)

    cnts,_ = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    areas =list()
    for c in cnts:
        area = cv2.contourArea(c)
        x,y,w,h = cv2.boundingRect(c)
        if area>800 and area<900:
            x,y,w,h = cv2.boundingRect(c)
            img[y:y+h,x:x+w]=0
    return img

In [ ]:
# This function removes the white dots in a straight line
def inpaint_dots(img):
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    thresholded = np.logical_and(*[lab[..., i] > t for i, t in enumerate([200, 0, 0])])
    thresholded=thresholded.astype('uint8')*255
    close_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    close = cv2.morphologyEx(thresholded.astype(np.uint8), cv2.MORPH_CLOSE, close_kernel, iterations=1)
    dilate_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (8,8))
    dilate = cv2.dilate(close, dilate_kernel, iterations=1)
    mask=np.zeros((img.shape[:-1]))
    cnts,_ = cv2.findContours(dilate.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    listx=[]
    listy=[]
    listw=[]
    listh=[]
    for cnt in cnts:
        x,y,w,h = cv2.boundingRect(cnt)
        listx.append(x)
        listy.append(y)
        listw.append(w)
        listh.append(h)

    roundedx=np.floor(np.array(listx)/10)*10
    counts=Counter(roundedx)
    indxs=np.where(roundedx==counts.most_common()[0][0])[0]
    listx=np.array(listx)[indxs]
    listy=np.array(listy)[indxs]
    listw=np.array(listw)[indxs]
    listh=np.array(listh)[indxs]

    if len(listx)>8: #### the circles are the majority
        for x_,y_,w_,h_ in zip(listx,listy,listw,listh):
            mask[y_:y_+h_,x_:x_+w_]=255
        img=cv2.inpaint(img.astype('uint8'),mask.astype('uint8'),4,cv2.INPAINT_TELEA)
    return img

In [ ]:
# Function call
pipeline = keras_ocr.pipeline.Pipeline()
preprocessed_image =inpaint_text(inpaint_dots(inpaint_P(crop_image(orig_image))),pipeline)